In [1]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider, DatetimeTickFormatter
from bokeh.io import output_file, show
import pandas as pd
import numpy as np

from bokeh.plotting import figure

import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (14,10)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
#Settings for plotting
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)

%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps


import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
locale.setlocale(locale.LC_ALL,"Danish")

# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

'Danish_Denmark.1252'

# Load data 

In [2]:
ssidatapath = "ssi_data"
rootdir = os.getcwd()[:-5] + ssidatapath
dfAdm = pd.read_excel('..\Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')
dfCase = pd.DataFrame(columns=dfAdm.columns)

In [3]:
for k in range(0,len(curDates)):
    d = curDates[k]
    curIndex = dfAdm.index[k]

    thisDateStr = d.strftime('%Y-%m-%d')
    curDir = rootdir + '/SSI_data_' + thisDateStr 
    curFilePath = curDir + '/Cases_by_age.csv'
    casedatadf = pd.read_csv(curFilePath,delimiter=';',dtype=str)

    casedatadf['Antal_bekræftede_COVID-19'] = pd.to_numeric(casedatadf['Antal_bekræftede_COVID-19'].astype(str).apply(lambda x: x.replace('.','')))

    dfCase.loc[curIndex] = casedatadf['Antal_bekræftede_COVID-19'].values

# Data: Collect the admission data in same style

In [4]:
dfAdmDiff = dfAdm.diff().iloc[1:]

dfAdmDiff['Dato'] = curDates[1:]

dfAdmDiffSum = dfAdmDiff.iloc[:,1:-2].cumsum(axis=1)
dfAdmDiffSum['Dato'] = curDates[1:]

dfAdmDiff = dfAdmDiff.iloc[:-1]
dfAdmDiffSum = dfAdmDiffSum.iloc[:-1]


dfAdmDiffSum['I alt'] = dfAdmDiffSum['90+']

# dfAdmDiffSum

# admdfData = pd.DataFrame()
# admdfData['Dato'] = curDates[1:]
# admdfData['Total'] = np.diff(dfAdm['I alt'].astype(int).values)
# admdfData = admdfData.iloc[:-1]
# admdfData 

In [5]:
# Same things, but with cases
dfCaseDiff = dfCase.diff().iloc[1:]

dfCaseDiff['Dato'] = curDates[1:]

# Bad hack to ignore 10-15 year olds!
dfCaseDiff['10-19'] = np.multiply(dfCaseDiff['10-19'],0.5)

dfCaseDiffSum = dfCaseDiff.iloc[:,1:-2].cumsum(axis=1)
dfCaseDiffSum['Dato'] = curDates[1:]

dfCaseDiff = dfCaseDiff.iloc[:-1]
dfCaseDiffSum = dfCaseDiffSum.iloc[:-1]


dfCaseDiffSum['I alt'] = dfCaseDiffSum['90+']


# Load calculations

In [6]:
datafilename = '../model_results_spread_prediction_var_vacc.csv'
df = pd.read_csv(datafilename)
df.End =  pd.to_datetime(df.End,format='%Y-%m-%d')

In [7]:
df['RT2'] = np.round(np.multiply(df.RT,10)).astype(int)

In [8]:
curdf = df[df.vacc1 == 0.75]

In [9]:
def cmaptohex(tup):
    (r,g,b,a) = tup
    return "#{:02x}{:02x}{:02x}".format(int(r*255),int(g*255),int(b*255))

# Nye smittetilfælde

In [10]:

# curdf['x'] = np.arange(0,len(curdf))
# curdf['y'] = np.arange(0,len(curdf))

casedf = pd.DataFrame()

casedf['Date'] = curdf.End.unique().astype('datetime64[D]')
numVals = len(curdf.End.unique())
firstWeek = 20
weekNums = np.arange(firstWeek,firstWeek+numVals)
casedf['Week'] = weekNums

 


allAges = df.Age.unique()

allRTs = df.RT2.unique()


casedf['Total'] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values.shape)
prevVals = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values.shape)

for curRT in allRTs:
    curColName = 'Total_'+str(curRT)
    casedf[curColName] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values.shape)



for curAge in allAges:
    newVals = curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Count.values 
    curVals = prevVals + newVals
    # casedf[curAge] = curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Count.values 
    casedf[curAge] = curVals
    prevVals = curVals

    casedf['Total'] += curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Count.values 

    for curRT in allRTs:
        curColName = curAge+'_'+str(curRT)
        casedf[curColName] = curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Count.values 
        casedf['Total_'+str(curRT)] += curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Count.values 


# # Remove everythings after september first
# casedf = casedf[casedf.Date <= np.datetime64('2021-09-01')]

# casedf

In [11]:

# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(casedf)

figW = 1000
figH = 600

curMaxY = 1.1*max(casedf.Total_17)
curMaxY = 30000
firstDate = np.datetime64('2021-05-18')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')


# testDates = np.arange(np.datetime64('2021-05-01'),np.datetime64('2021-05-30'))
# testData = np.multiply(np.ones((len(testDates),)),5000)
# plot.line(testDates,testData)

minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.3, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center')

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("cases.html",title='Nye smittetilfælde')

show(column(plot,slider))

None


In [12]:

# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(casedf)
sourceData = ColumnDataSource(dfCaseDiffSum)

figW = 1000
figH = 600

curMaxY = 1.1*max(casedf.Total_17)
curMaxY = 30000
firstDate = np.datetime64('2021-05-18')
firstDate = np.datetime64('2021-04-01')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge, source=sourceData,color=curColor)
    plot.circle('Dato', curAge, source=sourceData,color=curColor)

plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


# testDates = np.arange(np.datetime64('2021-05-01'),np.datetime64('2021-05-30'))
# testData = np.multiply(np.ones((len(testDates),)),5000)
# plot.line(testDates,testData)

minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.3, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center')

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("casesData.html",title='Nye smittetilfælde')

show(column(plot,slider))

None


# Hospitaliseringer

In [13]:
curdf['Hosp'] = np.multiply(curdf.Count,curdf.CaseHospRate)

<ipython-input-13-7d6623e11856>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curdf['Hosp'] = np.multiply(curdf.Count,curdf.CaseHospRate)


In [14]:

# curdf['x'] = np.arange(0,len(curdf))
# curdf['y'] = np.arange(0,len(curdf))

admdf = pd.DataFrame()

admdf['Date'] = curdf.End.unique().astype('datetime64[D]')
numVals = len(curdf.End.unique())
firstWeek = 20
weekNums = np.arange(firstWeek,firstWeek+numVals)
admdf['Week'] = weekNums

 


allAges = df.Age.unique()

allRTs = df.RT2.unique()


admdf['Total'] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Hosp.values.shape)
prevVals = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Hosp.values.shape)

for curRT in allRTs:
    curColName = 'Total_'+str(curRT)
    admdf[curColName] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Hosp.values.shape)



for curAge in allAges:
    newVals = curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Hosp.values 
    curVals = prevVals + newVals
    # admdf[curAge] = curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Hosp.values 
    admdf[curAge] = curVals
    prevVals = curVals

    admdf['Total'] += curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Hosp.values 

    for curRT in allRTs:
        curColName = curAge+'_'+str(curRT)
        admdf[curColName] = curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Hosp.values 
        admdf['Total_'+str(curRT)] += curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Hosp.values 


# # Remove everythings after september first
# admdf = admdf[admdf.Date <= np.datetime64('2021-09-01')]

admdf

,Date,Week,Total,Total_9,Total_10,Total_11,Total_12,Total_13,Total_14,Total_15,...,90+,90+_9,90+_10,90+_11,90+_12,90+_13,90+_14,90+_15,90+_16,90+_17
0,2021-05-18,20,285.560000,2.855600e+02,285.560000,285.560000,285.560000,285.560000,285.560000,285.560000,...,285.560000,2.800000e+00,2.800000e+00,2.800000e+00,2.800000e+00,2.800000e+00,2.800000e+00,2.800000e+00,2.800000e+00,2.800000e+00
1,2021-05-25,21,277.834282,2.170492e+02,232.959813,248.355191,263.296788,277.834282,292.008537,305.853681,...,277.834282,3.490490e-01,3.746358e-01,3.993940e-01,4.234224e-01,4.468010e-01,4.695954e-01,4.918606e-01,5.136429e-01,5.349822e-01
2,2021-06-01,22,279.080766,1.703235e+02,196.209604,222.999929,250.639413,279.080766,308.282831,338.209395,...,279.080766,4.349832e-02,5.010930e-02,5.695118e-02,6.400994e-02,7.127348e-02,7.873129e-02,8.637413e-02,9.419379e-02,1.021829e-01
3,2021-06-08,23,282.825322,1.349611e+02,166.832076,202.097840,240.761378,282.825322,328.292006,377.163517,...,282.825322,5.420519e-03,6.702059e-03,8.120517e-03,9.676079e-03,1.136892e-02,1.319918e-02,1.516702e-02,1.727258e-02,1.951597e-02
4,2021-06-15,24,282.737639,1.057055e+02,140.147025,180.861944,228.260127,282.737639,344.678076,414.453671,...,282.737639,6.754715e-04,8.963871e-04,1.157875e-03,1.462676e-03,1.813454e-03,2.212802e-03,2.663254e-03,3.167285e-03,3.727319e-03
5,2021-06-22,25,282.004500,8.288093e+01,117.771396,161.783938,216.119778,282.004500,360.684164,453.421728,...,282.004500,8.417301e-05,1.198899e-04,1.650970e-04,2.211039e-04,2.892633e-04,3.709690e-04,4.676535e-04,5.807863e-04,7.118725e-04
6,2021-06-29,26,276.104917,6.410023e+01,97.528564,142.454243,201.163403,276.104917,369.874436,485.196900,...,276.104917,1.048911e-05,1.603503e-05,2.354056e-05,3.342293e-05,4.614027e-05,6.219171e-05,8.211747e-05,1.064990e-04,1.359589e-04
7,2021-07-06,27,259.426100,4.787858e+01,77.914273,120.837002,180.068116,259.426100,363.087284,495.531254,...,259.426100,1.307087e-06,2.144651e-06,3.356559e-06,5.052342e-06,7.359816e-06,1.042623e-05,1.441939e-05,1.952875e-05,2.596647e-05
8,2021-07-13,28,225.678953,3.337137e+01,58.012678,95.373283,149.651362,225.678953,328.826387,464.845808,...,225.678953,1.628810e-07,2.868426e-07,4.785991e-07,7.637319e-07,1.173961e-06,1.747922e-06,2.531968e-06,3.580992e-06,4.959276e-06
9,2021-07-20,29,180.948259,2.160705e+01,40.084495,69.748338,114.985752,180.948259,273.344004,398.072744,...,180.948259,2.029720e-08,3.836460e-08,6.824163e-08,1.154487e-07,1.872580e-07,2.930333e-07,4.446001e-07,6.566476e-07,9.471607e-07


In [15]:
dfAdm = pd.read_excel('..\Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')


dfAdmDiff = dfAdm.diff().iloc[1:]

dfAdmDiff['Dato'] = curDates[1:]

dfAdmDiffSum = dfAdmDiff.iloc[:,1:-2].cumsum(axis=1)
dfAdmDiffSum['Dato'] = curDates[1:]

dfAdmDiff = dfAdmDiff.iloc[:-1]
dfAdmDiffSum = dfAdmDiffSum.iloc[:-1]


dfAdmDiffSum['I alt'] = dfAdmDiffSum['90+']
dfAdmDiffSum

# admdfData = pd.DataFrame()
# admdfData['Dato'] = curDates[1:]
# admdfData['Total'] = np.diff(dfAdm['I alt'].astype(int).values)
# admdfData = admdfData.iloc[:-1]
# admdfData 

Aldersgruppe,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,Dato,I alt
17_11_2020,4,17,35,61,110,165,228,270,284,2020-11-17,284
24_11_2020,1,17,36,64,113,163,222,292,303,2020-11-24,303
01_12_2020,1,8,20,52,97,130,193,252,267,2020-12-01,267
08_12_2020,7,30,62,103,164,231,341,421,442,2020-12-08,442
15_12_2020,14,41,89,148,250,366,486,589,625,2020-12-15,625
22_12_2020,15,57,127,222,365,517,753,938,992,2020-12-22,992
29_12_2020,12,60,110,208,356,526,811,1023,1084,2020-12-29,1084
05_01_2021,14,51,109,181,302,463,674,898,971,2021-01-05,971
12_01_2021,6,39,76,125,233,349,555,766,824,2021-01-12,824
19_01_2021,3,18,50,96,183,280,447,599,656,2021-01-19,656


In [21]:

from bokeh.models import DatetimeTickFormatter
# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(admdf)
sourceData = ColumnDataSource(dfAdmDiffSum)

figW = 1000
figH = 600

curMaxY = 1.1*max(admdf.Total_17)
# curMaxY = 30000

firstDate = np.datetime64('2021-05-18')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

# firstDate = np.datetime64('2021-04-01')
# lastDate = np.datetime64('2021-09-01')
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye hospitalindlæggelser'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



# for k in range(0,len(allAges)):
#     curAge = allAges[k]
#     curColor = cmaptohex(cmap(k))
#     plot.line('Dato', curAge, source=sourceData,color=curColor)
#     plot.circle('Dato', curAge, source=sourceData,color=curColor)

# plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
# plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.3, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center')

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("hosp.html",title='Hospitaliseringer')

show(column(plot,slider))

None


In [ ]:

from bokeh.models import DatetimeTickFormatter
# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(admdf)
sourceData = ColumnDataSource(dfAdmDiffSum)

figW = 1000
figH = 600

curMaxY = 1.1*max(admdf.Total_17)
# curMaxY = 30000

# firstDate = np.datetime64('2021-05-18')
# lastDate = np.datetime64('2021-09-01')
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

firstDate = np.datetime64('2021-04-01')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye hospitalindlæggelser'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge, source=sourceData,color=curColor)
    plot.circle('Dato', curAge, source=sourceData,color=curColor)

plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.3, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center')

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("hospData.html",title='Hospitaliseringer')

show(column(plot,slider))

# OLD

In [17]:

# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color='#00ff00')
# plot.line('Week', '20-29', source=source, line_width=3, line_alpha=0.6, color='#20dd20')
# plot.line('Week', '30-39', source=source, line_width=3, line_alpha=0.6, color='#307030')
# plot.line('Week', '40-49', source=source, line_width=3, line_alpha=0.6, color='#406040')
# plot.line('Week', '50-59', source=source, line_width=3, line_alpha=0.6, color='#505050')
# plot.line('Week', '60-69', source=source, line_width=3, line_alpha=0.6, color='#604060')
# plot.line('Week', '70-79', source=source, line_width=3, line_alpha=0.6, color='#703070')
# plot.line('Week', '80-89', source=source, line_width=3, line_alpha=0.6, color='#802080')
# plot.line('Week',   '90+', source=source, line_width=3, line_alpha=0.6, color='#901090')
# plot.line('Week', 'Total', source=source, line_width=3, line_alpha=0.6,color='black')

# plot.circle('Week', '10-19', source=source, color='#108000')
# plot.circle('Week', '20-29', source=source, color='#208000')
# plot.circle('Week', '30-39', source=source, color='#308000')
# plot.circle('Week', '40-49', source=source, color='#408000')
# plot.circle('Week', '50-59', source=source, color='#508000')
# plot.circle('Week', '60-69', source=source, color='#608000')
# plot.circle('Week', '70-79', source=source, color='#708000')
# plot.circle('Week', '80-89', source=source, color='#808000')
# plot.circle('Week',   '90+', source=source, color='#908000')
# plot.circle('Week', 'Total', source=source,color='black')
# plot.line('Week', 'Total', source=source, line_width=3, line_alpha=0.6)


# plot.varea('Week', 0 ,'10-19', source=source, fill_color=cmaptohex(cmap(1)))
# plot.varea('Week','10-19', '20-29', source=source, color=cmaptohex(cmap(2)))
# plot.varea('Week','20-29', '30-39', source=source, color=cmaptohex(cmap(3)))
# plot.varea('Week','30-39', '40-49', source=source, color=cmaptohex(cmap(4)))
# plot.varea('Week','40-49', '50-59', source=source, color=cmaptohex(cmap(5)))
# plot.varea('Week','50-59', '60-69', source=source, color=cmaptohex(cmap(6)))
# plot.varea('Week','60-69', '70-79', source=source, color=cmaptohex(cmap(7)))
# plot.varea('Week','70-79', '80-89', source=source, color=cmaptohex(cmap(8)))
# plot.varea('Week','80-89',   '90+', source=source, color=cmaptohex(cmap(9)))

# slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")

In [18]:
# curdf['RT2'] = np.round(np.multiply(curdf.RT,10)).astype(int)


    # for (var i = 0; i < y1.length; i++) {
    #     y1[i] = data[curName1][i]
    #     y2[i] = data[curName2][i]
    #     y3[i] = data[curName3][i]
    #     y4[i] = data[curName4][i]
    #     y5[i] = data[curName5][i]
    #     y6[i] = data[curName6][i]
    #     y7[i] = data[curName7][i]
    #     y8[i] = data[curName8][i]
    #     y9[i] = data[curName9][i]
    #     yT[i] = data[curNameT][i]
    # }


    # // Summing up
    # for (var i = 0; i < y1.length; i++) {
    #     y1[i] = data[curName1][i]
    #     y2[i] = data[curName2][i] + y1[i]
    #     y3[i] = data[curName3][i] + y2[i]
    #     y4[i] = data[curName4][i] + y3[i]
    #     y5[i] = data[curName5][i] + y4[i]
    #     y6[i] = data[curName6][i] + y5[i]
    #     y7[i] = data[curName7][i] + y6[i]
    #     y8[i] = data[curName8][i] + y7[i]
    #     y9[i] = data[curName9][i] + y8[i]
    #     yT[i] = data[curNameT][i]
    # }

In [19]:

# # x = [x*0.005 for x in range(0, 201)] 

# # source = ColumnDataSource(data=dict(x=x, y=x))
# # source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
# source = ColumnDataSource(curdf)

# plot = figure(plot_width=400, plot_height=400)
# plot.line('x', 'Count', source=source, line_width=3, line_alpha=0.6)

# # slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")
# minRT = 0.9
# maxRT = 1.7
# slider = Slider(start=minRT, end=maxRT, value=1.1, step=.1, title="RT")

# update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
#     var data = source.data;
#     var f = slider.value;
#     var age = data['Age']
#     var RT = data['RT2']
#     var End = data['End']
#     var Count = data['Count']

#     var x = data['x']
#     var y = data['y']

#     console.log(Count)
#     for (var i = 0; i < RT.length; i++) {
#         y[i] = 6
#         Count[i] = 0
        
#         if (RT[i] == Math.round(f*10)){
#             y[i] = data['Count'][i]
#             console.log(data['Count'][i])
#             Count[i] = data['Count'][i]

#             // End.push(data['End'][i])
#             // Count.push(data['Count'][i])
#         }

#     }

#     //for (var i = 0; i < x.length; i++) {
#         // y[i] = Math.pow(x[i], f)
#     //}
    
#     // necessary becasue we mutated source.data in-place
#     source.change.emit();
# """)
# slider.js_on_change('value', update_curve)

# output_file("plot.html")

# show(column(slider, plot))

In [20]:

# testdf['Total'] = curdf[(curdf.RT2 == 11)].Count.values 
# for curRT in allRTs:
#     curColName = 'Total_'+str(curRT)
#     testdf[curColName] = curdf[(curdf.RT2 == curRT)].Count.values 

# testdf['20_RT10'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 10)].Count.values
# testdf['20_RT11'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 11)].Count.values
# testdf['20_RT12'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values
# testdf['20_RT13'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values
# testdf['20_RT14'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 14)].Count.values
# testdf['RT11_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 11)].Count.values
# testdf['RT12_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values
# testdf['RT13_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values
# testdf['RT14_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 14)].Count.values

    # if (Math.round(f*10) == 11){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT11_20'][i]
    #     }
    # }
    # if (Math.round(f*10) == 12){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT12_20'][i]
    #     }
    # }
    # if (Math.round(f*10) == 13){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT13_20'][i]
    #     }
    # }
    
